# Negative-binomial data, Poisson calibration (robustness check)

We **break the Poisson assumption** by simulating true counts from a **negative binomial** (overdispersed). We then fit the **same** multi-site Poisson calibration model to see how robust it is to this misspecification.

Negative binomial: same mean \(\mu\) as Poisson but variance \(\mu + \mu^2/\phi\) (overdispersion when \(\phi\) is finite). As \(\phi \to \infty\) we recover Poisson.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import nbinom
from cmdstanpy import CmdStanModel

/Users/LOF19/miniconda3/envs/stan/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Simulate multi-site data (Negative Binomial true counts)

Same setup as the Poisson demo (sites, \(\alpha_s, \beta_s\), ragged sizes), but **true counts** are drawn from a negative binomial with mean 1.5 and dispersion \(\phi\) (smaller \(\phi\) = more overdispersion). Predictions are still generated as Poisson(exp(alpha + beta*log(true+eps))).

In [2]:
N_sites = 3
rng = np.random.default_rng(43)
epsilon = 1e-6

# Overdispersion: NB has var = mu + mu^2/phi. Smaller phi => more overdispersion.
phi = 3.0
base_mean = 1.5  # mean true count when present

# True (alpha, beta) per site
alpha_true_sites = np.array([0.2, 0.5, -0.1])
beta_true_sites = np.array([0.5, 0.8, 0.3])

# Total images per site (ragged)
N_images_per_site = np.array([200, 350, 150])
site_ids_full = np.repeat(np.arange(N_sites), N_images_per_site) + 1
N_total = len(site_ids_full)

# Simulate: true counts from NEGATIVE BINOMIAL (not Poisson)
# NB(phi, p) with p = phi/(phi+mu) gives mean mu, var mu + mu^2/phi
true_counts_multi = np.zeros(N_total, dtype=np.int64)
predicted_counts_multi = np.zeros(N_total)
idx = 0
for s in range(N_sites):
    n = N_images_per_site[s]
    presence = rng.binomial(1, 0.6, n)
    # Draw NB with mean base_mean; nbinom(n=phi, p=phi/(phi+mu)) -> mean mu, var mu+mu^2/phi
    nb_counts = nbinom.rvs(phi, phi / (phi + base_mean), size=n, random_state=rng)
    true_counts_multi[idx : idx + n] = presence * nb_counts
    for i in range(n):
        pred = np.exp(
            alpha_true_sites[s]
            + beta_true_sites[s] * np.log(true_counts_multi[idx + i] + epsilon)
        )
        predicted_counts_multi[idx + i] = rng.poisson(pred)
    idx += n

# Label a subset per site (ragged)
N_labeled_per_site = np.array([30, 80, 20])
labeled_indices = []
for s in range(N_sites):
    site_inds = np.where(site_ids_full == s + 1)[0]
    n_label = min(N_labeled_per_site[s], len(site_inds))
    chosen = rng.choice(site_inds, size=n_label, replace=False)
    labeled_indices.extend(chosen)
labeled_indices = np.array(labeled_indices)
unlabeled_mask = np.ones(N_total, dtype=bool)
unlabeled_mask[labeled_indices] = False

pred_labeled = predicted_counts_multi[labeled_indices].astype(np.float64)
truth_labeled = true_counts_multi[labeled_indices]
site_id_labeled = site_ids_full[labeled_indices].astype(np.int32)
pred_unlabeled = predicted_counts_multi[unlabeled_mask].astype(np.float64)
site_id_unlabeled = site_ids_full[unlabeled_mask].astype(np.int32)
truth_unlabeled = true_counts_multi[unlabeled_mask]

N_labeled = len(pred_labeled)
N_unlabeled = len(pred_unlabeled)
print(f"Data: NB(mean={base_mean}, phi={phi}) true counts. Sites: {N_sites}, Labeled: {N_labeled}, Unlabeled: {N_unlabeled}.")
print(f"True count variance / mean (overdispersion): {(true_counts_multi.var() / (true_counts_multi.mean() + 1e-10)):.2f}")

Data: NB(mean=1.5, phi=3.0) true counts. Sites: 3, Labeled: 130, Unlabeled: 570.
True count variance / mean (overdispersion): 2.01


## Fit the Poisson calibration model (misspecified)

We fit the **same** multi-site Poisson model as in the main demo. It assumes true counts are Poisson; the data are overdispersed.

In [3]:
data = {
    "N_sites": N_sites,
    "N_labeled": N_labeled,
    "N_unlabeled": N_unlabeled,
    "predicted_counts_labeled": pred_labeled,
    "true_counts_labeled": truth_labeled.tolist(),
    "site_id_labeled": site_id_labeled.tolist(),
    "predicted_counts_unlabeled": pred_unlabeled,
    "site_id_unlabeled": site_id_unlabeled.tolist(),
    "epsilon": epsilon,
}

model = CmdStanModel(stan_file="../stan_models/multi_site_count_calibration.stan")
fit = model.sample(data=data, seed=456, show_progress=True)

09:24:23 - cmdstanpy - INFO - CmdStan start processing
chain 1:   0%|          | 0/2000 [00:00<?, ?it/s, (Warmup)]


chain 1:  50%|█████     | 1000/2000 [00:00<00:00, 11374.23it/s, (Sampling)]


chain 1:  55%|█████▌    | 1100/2000 [00:00<00:00, 9464.07it/s, (Sampling)] 





chain 2: 100%|██████████| 2000/2000 [00:00<00:00, 4731.86it/s, (Sampling completed)]

chain 3: 100%|██████████| 2000/2000 [00:00<00:00, 4734.72it/s, (Sampling completed)]


chain 4: 100%|██████████| 2000/2000 [00:00<00:00, 4736.48it/s, (Sampling completed)]


09:24:23 - cmdstanpy - INFO - CmdStan done processing.


## Per-site calibration parameters

If the Poisson calibration is robust, we should still recover \(\alpha_s\) and \(\beta_s\) reasonably (possibly with wider CIs).

In [4]:
alpha_draws = fit.stan_variable("alpha")
beta_draws = fit.stan_variable("beta")
print("Per-site calibration (posterior mean [90% CI]); true in parentheses:")
for s in range(N_sites):
    a = alpha_draws[:, s]
    b = beta_draws[:, s]
    print(f"  Site {s+1}: alpha = {a.mean():.3f} [{np.percentile(a, 5):.3f}, {np.percentile(a, 95):.3f}]  (true {alpha_true_sites[s]:.3f})")
    print(f"           beta  = {b.mean():.3f} [{np.percentile(b, 5):.3f}, {np.percentile(b, 95):.3f}]  (true {beta_true_sites[s]:.3f})")

Per-site calibration (posterior mean [90% CI]); true in parentheses:
  Site 1: alpha = 0.715 [0.404, 1.009]  (true 0.200)
           beta  = 0.248 [0.162, 0.362]  (true 0.500)
  Site 2: alpha = 0.554 [0.357, 0.742]  (true 0.500)
           beta  = 0.249 [0.189, 0.319]  (true 0.800)
  Site 3: alpha = 0.615 [0.092, 1.101]  (true -0.100)
           beta  = 0.118 [0.059, 0.181]  (true 0.300)


## Maximum count per site (with CIs)

In [ ]:
max_count_draws = fit.stan_variable("max_count_site")
print("Per-site maximum rays in any image (posterior median and 90% CI):")
for s in range(N_sites):
    mc = max_count_draws[:, s]
    lo, hi = np.percentile(mc, [5, 95])
    print(f"  Site {s+1}: max = {int(np.median(mc))}  90% CI = [{int(lo)}, {int(hi)}]")
print("\nTrue max per site (from simulation):")
for s in range(N_sites):
    at_site = true_counts_multi[site_ids_full == s + 1]
    print(f"  Site {s+1}: {int(np.max(at_site))}")

## Overall and per-site mean rays per image

In [ ]:
mean_rays_overall = fit.stan_variable("mean_rays_per_image")
mean_rays_site = fit.stan_variable("mean_rays_per_image_site")
print("Overall mean rays per image:", mean_rays_overall.mean(), " 90% CI:", np.percentile(mean_rays_overall, [5, 95]))
print("True overall mean (simulation):", true_counts_multi.mean())
print("\nPer-site mean rays (posterior mean [90% CI]):")
for s in range(N_sites):
    ms = mean_rays_site[:, s]
    print(f"  Site {s+1}: {ms.mean():.3f} [{np.percentile(ms, 5):.3f}, {np.percentile(ms, 95):.3f}]")

## Posterior predictive coverage (unlabeled)

With overdispersed data, 90% intervals may be too narrow (under-coverage) because the Poisson model underestimates variance.

In [ ]:
rep = fit.stan_variable("true_counts_unlabeled_rep")
lo, hi = np.percentile(rep, [5, 95], axis=0)
covered = np.mean((truth_unlabeled >= lo) & (truth_unlabeled <= hi))
print(f"90% posterior predictive interval coverage of true unlabeled counts: {covered:.0%}")
print("(If data are overdispersed, coverage may drop below 90%.)")

## Plot: posterior of overall mean vs true mean

In [ ]:
mean_rays = fit.stan_variable("mean_rays_per_image")
lo, hi = np.percentile(mean_rays, [5, 95])
truth_overall = true_counts_multi.mean()

plt.figure(figsize=(7, 3))
plt.hist(mean_rays, bins=40, color="C0", alpha=0.7, edgecolor="white", density=True)
plt.axvline(mean_rays.mean(), color="C0", lw=2, label=f"Posterior mean = {mean_rays.mean():.3f}")
plt.axvline(lo, color="gray", ls="--", lw=1.5, label=f"90% CI = [{lo:.3f}, {hi:.3f}]")
plt.axvline(hi, color="gray", ls="--", lw=1.5)
plt.axvline(truth_overall, color="black", lw=2, label=f"True mean (NB data) = {truth_overall:.3f}")
plt.xlabel("Average rays per image")
plt.ylabel("Posterior density")
plt.title("Poisson calibration on NB data: posterior of overall mean")
plt.legend()
plt.tight_layout()
plt.show()